In [3]:
from itertools import chain
from typing import List, Tuple, Union

import pandas as pd

from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import BpeTrainer
from tokenizers.normalizers import Lowercase

import torch
import torch.nn as nn
import torch.optim as optim


import random
import re
from collections import Counter

import gensim.downloader as api
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from IPython.display import clear_output
from torch.nn.utils.rnn import pad_sequence
from torch.optim import Adam
from torch.optim import Optimizer
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm import tqdm

In [4]:
train = pd.read_parquet('/kaggle/input/nuclear-vk-bots/train.parquet')
test = pd.read_parquet('/kaggle/input/nuclear-vk-bots/test.parquet')

In [5]:
def clear(txt):
  txt = str(txt)
  txt = txt[1:]
  txt = txt.replace("'", "")
  txt = txt.replace("[", "")
  txt = txt.replace("]", "")
  txt = txt.replace(",", " ")
  txt = txt.replace('"', '')
  txt = txt.replace("\n", "")
  return txt

In [6]:
train['ciphers'] = train['ciphers'].apply(clear)
test['ciphers'] = test['ciphers'].apply(clear)
train['curves'] = train['ciphers'].apply(clear)
test['curves'] = test['curves'].apply(clear)

In [7]:
train['text'] = train.apply(lambda row: f"{row.ciphers} [SEP] {row.curves} [SEP] {row.ua}", axis=1)

In [8]:
test['text'] = test.apply(lambda row: f"{row.ciphers} [SEP] {row.curves} [SEP] {row.ua}", axis=1)

In [9]:
train_new = train.drop(["ciphers", "curves", "ua"], axis = 1)

test_new = test.drop(["ciphers", "curves", "ua"], axis = 1)

In [10]:
from transformers import BertForSequenceClassification # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertForSequenceClassification
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaForSequenceClassification
from transformers import DistilBertModel
from transformers import GPT2Model
from transformers import BertModel

In [11]:
from transformers import AutoTokenizer

old_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")
new_tokenizer = old_tokenizer.train_new_from_iterator([f"{row.ciphers} [SEP] {row.curves} [SEP] {row.ua}" for row in chain(train.itertuples(), test.itertuples())], 
                                                   vocab_size= 15000)
new_tokenizer.save_pretrained("/kaggle/working/bruhwalkk")

('/kaggle/working/bruhwalkk/tokenizer_config.json',
 '/kaggle/working/bruhwalkk/special_tokens_map.json',
 '/kaggle/working/bruhwalkk/vocab.txt',
 '/kaggle/working/bruhwalkk/added_tokens.json',
 '/kaggle/working/bruhwalkk/tokenizer.json')

In [12]:
from transformers import DistilBertTokenizer

new_tokenizer = DistilBertTokenizer.from_pretrained("/kaggle/working/bruhwalkk")
new_tokenizer

DistilBertTokenizer(name_or_path='/kaggle/working/bruhwalkk', vocab_size=10996, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [13]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_new)
test_data = Dataset.from_pandas(test_new)

In [14]:
def tokenization(example):
    return new_tokenizer.batch_encode_plus(example['text'], add_special_tokens=True, return_token_type_ids=False, truncation=True)

train_dataset = train_data.map(tokenization, batched=True)
test_dataset = test_data.map(tokenization, batched=True)

  0%|          | 0/48 [00:00<?, ?ba/s]

  0%|          | 0/15 [00:00<?, ?ba/s]

In [15]:
train_dataset.set_format(type="torch", columns=["id", "input_ids", "attention_mask", "label"])
test_dataset.set_format(type="torch", columns=["id", "input_ids", "attention_mask"])

In [16]:
from transformers import DataCollatorWithPadding


data_collator = DataCollatorWithPadding(tokenizer=new_tokenizer)

In [17]:
from torch.utils.data import DataLoader


train_loader = DataLoader(train_dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [18]:
from transformers import DistilBertConfig

config = DistilBertConfig(
    vocab_size=new_tokenizer.vocab_size,  # we align this to the tokenizer vocab_size
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [19]:
from transformers import DistilBertForSequenceClassification

mod = DistilBertForSequenceClassification(config)

In [20]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)
print(torch.cuda.get_device_name())

cuda:0
Tesla P100-PCIE-16GB


In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [28]:
from transformers import DistilBertForSequenceClassification

mod2 = DistilBertForSequenceClassification.from_pretrained("bruhwalkk/distilbert-base-uncased-finetuned-vk")

Some weights of the model checkpoint at bruhwalkk/distilbert-base-uncased-finetuned-vk were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at bruhwalkk/distilbert-base-uncased-finetuned-vk and are newly initialized: ['pre_clas

In [29]:
from transformers import TrainingArguments, Trainer

mod2 = mod2.to(device)

training_args = TrainingArguments(
    save_steps = 5000,
    logging_strategy = "epoch",
    report_to="none",
    output_dir="distilbert-base-uncased-finetuned-vk-classification_last",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=6,
    push_to_hub = True,
    weight_decay=0.01
)

trainer = Trainer(
    model=mod2,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=new_tokenizer,
    data_collator=data_collator
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/kaggle/working/distilbert-base-uncased-finetuned-vk-classification_last is already a clone of https://huggingface.co/bruhwalkk/distilbert-base-uncased-finetuned-vk-classification_last. Make sure you pull the latest changes with `repo.git_pull()`.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Step,Training Loss
2997,0.360700
5994,0.319600
8991,0.294800
11988,0.278800
14985,0.262300
17982,0.249600


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

TrainOutput(global_step=17982, training_loss=0.29431081743635995, metrics={'train_runtime': 7008.4914, 'train_samples_per_second': 41.048, 'train_steps_per_second': 2.566, 'total_flos': 3.4377237894678252e+16, 'train_loss': 0.29431081743635995, 'epoch': 6.0})

In [30]:
pred, _ , _ = trainer.predict(test_dataset)
pred = torch.sigmoid(torch.tensor(pred))

In [31]:
test_res = pd.DataFrame({
        "id": test_dataset["id"].numpy(),
        "is_bot": pred[:, 1].numpy()})

In [33]:
test_res.to_csv("submission_bert_moment-tuned-best?.csv", index=None)

In [ ]:
pred2, _, _ = trainer.predict(train_dataset)
pred2 = torch.sigmoid(torch.tensor(pred2))

In [ ]:
train_res = pd.DataFrame({
        "id": train_dataset["id"].numpy(),
        "is_bot": pred2[:, 1].numpy(), 
    "label": train_dataset["label"].numpy()})

In [ ]:
from sklearn.metrics import roc_auc_score

print(f"ROC-AUC SCORE FOR TRAIN IS: {roc_auc_score(train_res["label"], train_res["is_bot"])}")

In [ ]:
trainer.push_to_hub("distilbert-base-uncased-finetuned-vk-classification_last")